In [2]:
import bob
import bob.bio.gmm

ModuleNotFoundError: No module named 'bob'

In [1]:
import numpy as np
import librosa

import matlab
import matlab.engine

from IPython.display import Audio 

%matplotlib inline
import matplotlib.pyplot as plt

import scattering

##### Load signal

In [2]:
signal_path = "/home/julia/DeepVoice_data/samples/clear.wav"
signal_wav, sr = librosa.load(signal_path)

print(sr)
Audio(signal_wav, rate = sr)

22050


##### Create batch of signals and count scattering coefficients using matlab scattering implementation

function get_scattering_coefficients() from custom module scattering.py is used

In [3]:
2**(-58/8) - (2*np.pi*8)/8192

0.00043358009262707987

In [4]:
1024*16

16384

0.2048

In [3]:
# N = 40000
# T = 4096*2
# np_arr_batch = np.random.normal(size = (2, N))

np_arr_batch = np.array([signal_wav[:40000]]*2)
N = np_arr_batch.shape[1]
# T = 1024
T = 8192


path_to_scatnet = '/home/julia/Downloads/scatnet-master/'

for Q1 in [1]:
    S_table_batch, time_count, coeffs_count = scattering.get_scattering_coefficients(path_to_scatnet,
                                                                         np_arr_batch, N, T,
                                                                         M = 1, Q = [Q1],
                                                                         renorm = False,
                                                                         log = False)

    print(S_table_batch.shape, S_table_batch[0].shape)
    print(time_count, coeffs_count)

J = {'filter_type': 'morlet_1d', 'Q': 1.0, 'B': 1.0, 'xi_psi': 1.1547005383792517, 'sigma_psi': 1.1547005383792517, 'phi_bw_multiplier': 2.0, 'sigma_phi': 0.5773502691896258, 'J': 13.0, 'P': 0.0, 'precision': 'double', 'filter_format': 'fourier_truncated', 'boundary': 'symm', 'phi_dirac': 0.0}
{'bandwidth': 0.0007669903939428206, 'resolution': 12.0, 'j': matlab.double([]), 'q': matlab.double([])}
{'bandwidth': matlab.double([[0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206,0.0007669903939428206]]), 'resolution': matlab.double([[12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0]]), 'j': matlab.double([[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0]]), 'q': matlab.double([])}
(2, 14, 10) (14, 10)
10 [1.0, 13.0]


In [4]:
# J = 10

N = 40000
T = 8192

for Q in [1, 8]:
    print('\n ---------------- Q = {} ----------------'.format(Q))
    B = Q
    phi_bw_mult = 2 if Q == 1 else 1
    
    J = 1 + np.ceil(np.log2(T*phi_bw_mult/4/B))*Q
    print('J = {}'.format(J))
    
    xi_psi = []
    sigma_psi = []
    # Calculate logarithmically spaced band-pass filters (wavelets)
    # Centers of wavelets
    print('\n Wavelet log-scale filters:')
    for xi_psi_def in [np.pi/2*(1 + 2**(-1/Q))]:
        xi_psi_log = xi_psi_def*2**(-np.arange(0, J)[::-1]/Q)
        
        print('mother center = {},\ncenters = '.format(xi_psi_def), xi_psi_log)

    # Standard deviations of wavelets
    sigma0 = 2/np.sqrt(3)
    sigma_psi_def = 0.5*sigma0/(1 - 2**(-1/B))
    sigma_psi_log = sigma_psi_def*2**(np.arange(0, J)[::-1]/Q)
    print('sigma0 = {}'.format(sigma0))
    print('standard deviations = {}'.format(sigma_psi_log))

    # Calculate linearly spaced, band-pass filters so that they
    # evenly cover the remaining part of the spectrum

    sigma_phi_def = sigma_psi_def/phi_bw_mult
    P_def = np.ceil(2**(-1/Q) - 0.25*sigma0/sigma_phi_def/(1 - 2**(-1/Q)))
    
    step = np.pi * 2**(-J/Q) * (1 - 0.25*sigma0/sigma_phi_def * 2**(1/Q)) / P_def
    print('P = {}'.format(P_def))
    print('Step = {}'.format(step))
    
    xi_psi_linear = xi_psi_def*2**(-(J-1)/Q) - step * np.arange(1, P_def+1)
    sigma_psi_linear = [sigma_psi_def*2**((J-1)/Q)]*int(P_def)
    print('\n Wavelet linear-scale filters:')
    print('centers = {}'.format(xi_psi_linear))
    print('standard deviations = {}'.format(sigma_psi_linear))
    
    xi_psi.extend(xi_psi_linear)
    xi_psi.extend(xi_psi_log)
    sigma_psi.extend(sigma_psi_linear) 
    sigma_psi.extend(sigma_psi_log)
       
    
    # Calculate low-pass filter
    sigma_phi = sigma_phi_def*2**((J-1)/Q)
    print('\n Low-pass filter')
    print('standard deviation = {}\n'.format(sigma_phi))
    
    # Convert (spacial) signals to (frequential) bandwidth
    print('\n Convert (spacial) signals to (frequential) bandwidth')
    bw_psi = np.pi/2 * sigma0/sigma_psi
    print('Bandwidths of wavelet filters = {}\n'.format(bw_psi))
    
    bw_phi = np.pi/2*sigma0/sigma_phi
    print('Bandwidths of the lowpass filters = {}\n'.format(bw_phi))
    
    


 ---------------- Q = 1 ----------------
J = 13.0

 Wavelet log-scale filters:
mother center = 2.356194490192345,
centers =  [5.75242795e-04 1.15048559e-03 2.30097118e-03 4.60194236e-03
 9.20388473e-03 1.84077695e-02 3.68155389e-02 7.36310778e-02
 1.47262156e-01 2.94524311e-01 5.89048623e-01 1.17809725e+00
 2.35619449e+00]
sigma0 = 1.1547005383792517
standard deviations = [4.72965341e+03 2.36482670e+03 1.18241335e+03 5.91206676e+02
 2.95603338e+02 1.47801669e+02 7.39008345e+01 3.69504172e+01
 1.84752086e+01 9.23760431e+00 4.61880215e+00 2.30940108e+00
 1.15470054e+00]
P = -0.0
Step = nan

 Wavelet linear-scale filters:
centers = []
standard deviations = []

 Low-pass filter
standard deviation = 2364.8267026007074


 Convert (spacial) signals to (frequential) bandwidth
Bandwidths of wavelet filters = [3.83495197e-04 7.66990394e-04 1.53398079e-03 3.06796158e-03
 6.13592315e-03 1.22718463e-02 2.45436926e-02 4.90873852e-02
 9.81747704e-02 1.96349541e-01 3.92699082e-01 7.85398163e-01
 1.57

/home/julia/anaconda3/envs/bob_py3/lib/python3.6/site-packages/ipykernel/__main__.py:37: RuntimeWarning: invalid value encountered in double_scalars


##### Lets count scattering transform on frquency axis

In [5]:
np_arr_batch_freq = S_table_batch[0, -356:, :].T
N_freq = np_arr_batch_freq.shape[1]
T_freq = N_freq//10

print(np_arr_batch_freq.shape, T_freq)

S_table_batch_freq, freq_count, coeffs_count = scattering.get_scattering_coefficients(path_to_scatnet,
                                                                                      np_arr_batch_freq, N_freq, T_freq,
                                                                         M = 1, Q = [1],
                                                                         renorm = False,
                                                                         log = False)


(10, 14) 1
J = {'filter_type': 'morlet_1d', 'Q': 1.0, 'B': 1.0, 'xi_psi': 1.1547005383792517, 'sigma_psi': 1.1547005383792517, 'phi_bw_multiplier': 2.0, 'sigma_phi': 0.5773502691896258, 'J': 0.0, 'P': 0.0, 'precision': 'double', 'filter_format': 'fourier_truncated', 'boundary': 'symm', 'phi_dirac': 0.0}
{'bandwidth': 6.283185307179586, 'resolution': 0.0, 'j': matlab.double([]), 'q': matlab.double([])}
{'bandwidth': matlab.double([]), 'resolution': matlab.double([]), 'j': matlab.double([[]])}


In [ ]:
print(S_table_batch_freq.shape)
print(freq_count, coeffs_count)

##### Visualize scattering coefficients of all orders as well as of each order separately

In [ ]:
plt.figure(figsize = (10,5))
plt.imshow(S_table_batch[0])
plt.show()

print(S_table_batch[0].shape)
_, axs = plt.subplots(1, 3)
axs[0].imshow(S_table_batch[0][:1, :])
axs[1].imshow(S_table_batch[0][1:int(coeffs_count[1]), :])
axs[2].imshow(S_table_batch[0][int(coeffs_count[1]):, :])
plt.show()

# Playground

some examples that show how to use matlab implementation of scattering

In [ ]:
# run matlab engine
eng = matlab.engine.start_matlab()

# add path to matlab scattering implementation
path_to_scatnet = '/home/julia/Downloads/scatnet-master/'
eng.eval("addpath {}; addpath_scatnet;".format(path_to_scatnet), nargout = 0)

In [ ]:
# load signal from matlab library and set scattering parameters
eng.eval("load handel; N = length(y);", nargout = 0)
eng.eval("T = 2^12;", nargout = 0)
# eng.eval("filt_opt = default_filter_options('audio', T);", nargout = 0)
eng.eval("filt_opt.Q = {};".format(str([8, 1])), nargout = 0)
eng.eval("filt_opt.J = T_to_J(T, filt_opt);", nargout = 0)

# create filter bank
eng.eval("[Wop, filters] = wavelet_factory_1d(N, filt_opt)", nargout = 0)

# count scattering coefficients using filter bank, normalize and take logarithm (if needed)
eng.eval('S = scat(y, Wop);', nargout = 0)
eng.eval("S = renorm_scat(S);", nargout = 0)
eng.eval("S = log_scat(S);", nargout = 0)

# copy matlab variable to python variable
S = eng.eval('S;')

# collect all scattering coefficients in one table
eng.eval("[S_table, meta] = format_scat(S);", nargout = 0)

S_table = eng.eval('S_table;')

In [ ]:
print(eng.eval('size(S_table);'))

for i in range(3):
    print(np.asarray(S[i]['signal']).shape,
          np.asarray(S[i]['meta']['j']).shape,
          np.asarray(S[i]['meta']['bandwidth']).shape,
          np.asarray(S[i]['meta']['resolution']).shape)

In [ ]:
eng.eval('filt_opt')['J'], eng.eval('filt_opt')['Q']

In [ ]:
# count number of scatttering coefficients of each order
coeffs_count = []
for i in range(1,4):

    coeffs_shape_i = np.array(S[i-1]['signal']).shape
    coeffs_count_i = eng.eval("size(S{}.signal, 2);".format('{'+str(i)+'}'))
    time_count_i = eng.eval("size(S{}.signal{}, 1);".format('{'+str(i)+'}','{'+str(1)+'}'))
    
    coeffs_count.append(coeffs_count_i)

    print('Order {}:'.format(i), coeffs_shape_i, coeffs_count_i, time_count_i)

print(coeffs_count, '\t', sum(coeffs_count), 'coefficients in total')

print('\n Scattering table shape: {}'.format(np.array(S_table).shape))

##### Inverse scattering

not very fast, actually I haven't run nice experiments with it yet

In [ ]:
# can take couple of minutes
eng.eval("[xt, Ut] = inverse_scat(S, filters);", nargout = 0)

In [ ]:
xt = eng.eval('xt;')
y = eng.eval('y;')

In [ ]:
import IPython.display as ipd
import matplotlib.pylab as plt
% matplotlib inline

plt.plot(xt)
plt.show()

plt.plot(y)
plt.show()

In [ ]:
ipd.Audio(np.asarray(y).ravel(), rate = 8000)

In [ ]:
ipd.Audio(np.asarray(xt).ravel(), rate = 8000)